<a href="https://colab.research.google.com/github/arashk1990/covid19mobility/blob/main/survey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
def read_tab(var):
  dir = f'/content/drive/MyDrive/socio_pred/UKDA-5340-tab/tab/{var}_eul_2002-2020.tab'
  df = pd.read_csv(dir,sep='\t')
  #select surveys after 2018
  if 'SurveyYear' in df.columns:
    df = df[df['SurveyYear']>2018]
  return df

In [ ]:
#indev = read_tab('indev')
#stage = read_tab('stage')

In [3]:
#select the relevant vars
#PSU
psu = read_tab('psu')
psu = psu[['PSUID','SurveyYear','PSUStatsReg_B01ID']]

In [4]:
#Trip
trip = read_tab('trip')

cols_trip = ['TripID','DayID','IndividualID','HouseholdID','PSUID',\
             'SeriesCall_B01ID','ShortWalkTrip_B01ID','NumStages_B01ID',\
             'MainMode_B04ID','TripPurpose_B04ID','TripStart_B01ID',\
             'TripStart_B02ID','TripEnd_B01ID','TripEnd_B02ID',\
             'TripDisIncSW_B01ID','TripDisExSW_B01ID','TripTotalTime_B01ID',\
             'TripTravTime_B01ID','TripOrigGOR_B02ID','TripDestGOR_B02ID']

trip = trip[cols_trip]  
trip.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (37,40,43,45,46) have mixed types.Specify dtype option on import or set low_memory=False.
  


(277460, 20)

In [5]:
#filter trips
# no shortwalk trips 
trip = trip[trip['ShortWalkTrip_B01ID']==2]

In [6]:
#Household
household = read_tab('household')

cols_houshold = ['HouseholdID','PSUID','TWSDay','TWSMonth','TWSYear',\
                 'TWSMonth_B01ID','TWSWeekday_B01ID','TWEMonth_B01ID',\
                 'TWEWeekday_B01ID','HHIncome2002_B02ID','Ten1_B02ID',\
                 'HHoldGOR_B02ID','HHoldStruct_B02ID','HRPWorkStat_B02ID',\
                 'HRPEmpStat_B01ID','HRPSEGWorkStat_B01ID','HHoldEmploy_B01ID',
                 'WalkBus_B01ID','WalkRail_B01ID','BusRail_B01ID','Settlement2011EW_B03ID',\
                 'HHoldOAClass2011_B03ID','HHIncQDS2020Eng_B01ID','HHIncQIS2020Eng_B01ID']
household = household[cols_houshold]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (17,34,35,36,37,39,44,49,50,89) have mixed types.Specify dtype option on import or set low_memory=False.
  


In [ ]:
household.shape

(9839, 24)

In [7]:
# household filters:
# exclude 'DEAD' rows
household = household[household['TWSMonth_B01ID']>0]
#Nov, Dec 2019, Jan and Feb 2020
household=household[((household['TWSMonth_B01ID'] <3) & (household['TWSYear']==2020))\
          | ((household['TWSMonth_B01ID'] >10) & (household['TWSYear']==2019))]

household.shape

(2192, 24)

In [8]:
#trip filter
# is from selected hh 
trip = trip[trip['HouseholdID'].isin(household['HouseholdID'])]
trip.shape

#filter hou

(66252, 20)

In [9]:
#Individual
individual = read_tab('individual')

cols_ind = ['IndividualID','HouseholdID','PSUID','VehicleID','Age_B04ID',
 'Sex_B01ID','MarStat_B01ID','EthGroupTS_B02ID','EdAttn3_B01ID',
 'CarAccess_B01ID','DrivLic_B02ID','IndIncome2002_B02ID',\
 'WkPlace_B01ID','EcoStat_B02ID','SC_B01ID','OftHome_B01ID','BusOut_B01ID',\
 'Educ_B01ID','WkMuch_B01ID','OwnPhone_B01ID']
individual = individual[cols_ind]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (57) have mixed types.Specify dtype option on import or set low_memory=False.
  


In [10]:
individual.shape

(22767, 20)

In [11]:
#individual filters
# is from selected hh 
individual = individual[individual['HouseholdID'].isin(household['HouseholdID'])]
individual.shape

(5240, 20)

In [12]:
#Vehicle
vehicle = read_tab('vehicle')

cols_veh = ['VehicleID','HouseholdID'	,'PSUID'	,'IndividualID','VehNo',\
            'VehAvail_B01ID','VehComMile_B01ID','VehBusMile_B01ID',\
             'VehPriMile_B01ID']
vehicle = vehicle[cols_veh]

# is from selected hh 

vehicle = vehicle[vehicle['HouseholdID'].isin(household['HouseholdID'])]


In [13]:
#Day
day = read_tab('day')

cols_day = ['DayID','IndividualID','HouseholdID','PSUID','TravelWeekDay_B01ID',\
            'TravelWeekDay_B02ID','TravelWeekDay_B03ID','TravelDayType_B01ID']
day = day[cols_day]
# is from selected hh 
day = day[day['HouseholdID'].isin(household['HouseholdID'])]
day.shape

(32949, 8)

In [14]:
#merge with day 
trip_day = trip.merge(day, left_on='DayID', right_on='DayID', how='inner',suffixes=('', '_y'))

trip_day.drop(trip_day.filter(regex='_y$').columns, axis=1, inplace=True)


In [15]:
#merge with individual
trip_day_ind = trip_day.merge(individual, left_on='IndividualID', right_on='IndividualID', how='inner',suffixes=('', '_y'))

trip_day_ind.drop(trip_day_ind.filter(regex='_y$').columns, axis=1, inplace=True)

In [16]:
#merge with hh
trip_day_ind_hh = trip_day_ind.merge(household, left_on='HouseholdID', right_on='HouseholdID', how='inner',suffixes=('', '_y'))

trip_day_ind_hh.drop(trip_day_ind_hh.filter(regex='_y$').columns, axis=1, inplace=True)

In [17]:
#merge with PSU
trip_day_ind_hh_psu = trip_day_ind_hh.merge(psu, left_on='PSUID', right_on='PSUID', how='inner',suffixes=('', '_y'))

trip_day_ind_hh_psu.drop(trip_day_ind_hh_psu.filter(regex='_y$').columns, axis=1, inplace=True)

In [18]:
#merge with vehicle
all_tabs = trip_day_ind_hh_psu.merge(vehicle, left_on='VehicleID', right_on='VehicleID', how='inner',suffixes=('', '_y'))

all_tabs.drop(all_tabs.filter(regex='_y$').columns, axis=1, inplace=True)

In [19]:
#drop irrelivant columns
#to keep things simple, only some of the vars are kept
all_tabs.drop(columns=['DayID','ShortWalkTrip_B01ID','HouseholdID','PSUID',\
                       'SeriesCall_B01ID','MainMode_B04ID','TripPurpose_B04ID',\
                       'TripStart_B02ID','TripEnd_B02ID','TripOrigGOR_B02ID',\
                       'TripDestGOR_B02ID','TravelWeekDay_B02ID','TravelDayType_B01ID',\
                       'VehicleID','TWEWeekday_B01ID','TWSDay', 'TWSMonth',\
                       'TWSYear','TWSWeekday_B01ID','HHoldGOR_B02ID',
                       'WalkBus_B01ID', 'WalkRail_B01ID','BusRail_B01ID',\
                       'VehNo', 'VehComMile_B01ID','VehBusMile_B01ID',\
                       'VehPriMile_B01ID','NumStages_B01ID'],inplace=True)

In [20]:
all_tabs.columns

Index(['TripID', 'IndividualID', 'TripStart_B01ID', 'TripEnd_B01ID',
       'TripDisIncSW_B01ID', 'TripDisExSW_B01ID', 'TripTotalTime_B01ID',
       'TripTravTime_B01ID', 'TravelWeekDay_B01ID', 'TravelWeekDay_B03ID',
       'Age_B04ID', 'Sex_B01ID', 'MarStat_B01ID', 'EthGroupTS_B02ID',
       'EdAttn3_B01ID', 'CarAccess_B01ID', 'DrivLic_B02ID',
       'IndIncome2002_B02ID', 'WkPlace_B01ID', 'EcoStat_B02ID', 'SC_B01ID',
       'OftHome_B01ID', 'BusOut_B01ID', 'Educ_B01ID', 'WkMuch_B01ID',
       'OwnPhone_B01ID', 'TWSMonth_B01ID', 'TWEMonth_B01ID',
       'HHIncome2002_B02ID', 'Ten1_B02ID', 'HHoldStruct_B02ID',
       'HRPWorkStat_B02ID', 'HRPEmpStat_B01ID', 'HRPSEGWorkStat_B01ID',
       'HHoldEmploy_B01ID', 'Settlement2011EW_B03ID', 'HHoldOAClass2011_B03ID',
       'HHIncQDS2020Eng_B01ID', 'HHIncQIS2020Eng_B01ID', 'SurveyYear',
       'PSUStatsReg_B01ID', 'VehAvail_B01ID'],
      dtype='object')

Dealing with Missing values

In [21]:
#columns with negative values(missing in NTS data)
missing_cols = all_tabs[all_tabs<=0].any()[all_tabs[all_tabs<=0].any()==True].index
missing_cols

Index(['TripStart_B01ID', 'TripEnd_B01ID', 'EdAttn3_B01ID', 'WkPlace_B01ID',
       'EcoStat_B02ID', 'SC_B01ID', 'OftHome_B01ID', 'BusOut_B01ID',
       'Educ_B01ID', 'WkMuch_B01ID', 'OwnPhone_B01ID', 'Ten1_B02ID',
       'HHoldStruct_B02ID', 'HRPWorkStat_B02ID', 'HRPEmpStat_B01ID',
       'HHoldOAClass2011_B03ID', 'HHIncQDS2020Eng_B01ID',
       'HHIncQIS2020Eng_B01ID'],
      dtype='object')

In [22]:
for c in missing_cols:
  print(all_tabs[c].value_counts())

 16    3873
 9     3804
 17    3644
 18    3470
 12    3421
 13    3328
 11    3295
 15    3224
 14    2863
 10    2744
 19    2568
 8     2476
 20    1740
 21    1043
 7      935
 22     766
-8      674
 23     561
 6      358
 24     327
 5       96
 1       80
 2       37
 3       24
 4       12
Name: TripStart_B01ID, dtype: int64
 9     3937
 16    3846
 18    3546
 13    3364
 17    3358
 12    3320
 11    3266
 15    3117
 19    2989
 10    2965
 14    2896
 20    2214
 8     1709
 21    1210
 22     819
-8      747
 23     614
 7      573
 24     398
 6      215
 1      125
 2       50
 5       48
 3       19
 4       18
Name: TripEnd_B01ID, dtype: int64
-9    45363
Name: EdAttn3_B01ID, dtype: int64
 1    22676
-9    13808
 3     4881
 2     2262
 4     1724
-8       12
Name: WkPlace_B01ID, dtype: int64
 1    23868
 4    11026
 2     7687
 6     2169
 5      407
 3      201
-9        5
Name: EcoStat_B02ID, dtype: int64
 2    18729
 3     8426
 4     6516
 5     5494
 1     3262


In [23]:
#vars with many missing data
miss_all = ['EdAttn3_B01ID','WkPlace_B01ID','OftHome_B01ID','BusOut_B01ID',\
            'Educ_B01ID','WkMuch_B01ID','HHoldOAClass2011_B03ID',
            'HHIncQDS2020Eng_B01ID','HHIncQIS2020Eng_B01ID','OwnPhone_B01ID']

all_tabs.drop(columns=miss_all,inplace=True)
#vars with few missing data
miss_few = ['TripStart_B01ID','TripEnd_B01ID','EcoStat_B02ID','SC_B01ID','Ten1_B02ID',\
            'HHoldStruct_B02ID',
            'HRPWorkStat_B02ID','HRPEmpStat_B01ID']
for c in miss_few:
  all_tabs = all_tabs[all_tabs[c]>0]

#remaining after deleting missing values
all_tabs.shape


(41839, 32)

Dealing with categorical vars

In [24]:
noncat_cols = ['TripID', 'IndividualID','SurveyYear']
all_tabs_dummy = all_tabs 
for c in all_tabs.columns:
  if c not in noncat_cols:
    c_dummy = pd.get_dummies(all_tabs[c],prefix=c)
    all_tabs_dummy = pd.concat([all_tabs_dummy,c_dummy], axis=1)
    all_tabs_dummy.drop(columns = c,inplace=True)


In [25]:
ind_gp = all_tabs_dummy.groupby('IndividualID')
individual_tab = pd.DataFrame()
individual_tab ['Individual ID'] = ind_gp.size().index
individual_tab ['Total Number of Trips'] = ind_gp.size().values
mobility_sum = pd.DataFrame(ind_gp.sum().iloc[:,2:111]).reset_index().iloc[:,1:]
socio = ind_gp.mean().iloc[:,111:].reset_index().iloc[:,1:]
ids = ind_gp.mean().iloc[:,0:2].reset_index().iloc[:,1:]

NTS_dummy = pd.concat([individual_tab,ids,mobility_sum,socio], axis=1)



In [26]:
socio.shape

(2445, 99)

In [ ]:
NTS_dummy

,Individual ID,Total Number of Trips,TripID,SurveyYear,TripStart_B01ID_1,TripStart_B01ID_2,TripStart_B01ID_3,TripStart_B01ID_4,TripStart_B01ID_5,TripStart_B01ID_6,...,PSUStatsReg_B01ID_6,PSUStatsReg_B01ID_7,PSUStatsReg_B01ID_8,PSUStatsReg_B01ID_9,PSUStatsReg_B01ID_10,PSUStatsReg_B01ID_11,PSUStatsReg_B01ID_12,PSUStatsReg_B01ID_13,VehAvail_B01ID_1,VehAvail_B01ID_2
0,2019011363,16,2.019149e+09,2019.0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2019011428,18,2.019150e+09,2019.0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2019011564,23,2.019152e+09,2019.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,2019011569,15,2.019152e+09,2019.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,2019011570,13,2.019152e+09,2019.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2440,2020001977,30,2.020026e+09,2020.0,4,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2441,2020002004,15,2.020026e+09,2020.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2442,2020002009,8,2.020026e+09,2020.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2443,2020002010,6,2.020026e+09,2020.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [ ]:
NTS_dummy.to_csv('/content/drive/MyDrive/socio_pred/UKDA-5340-tab/NTS_dummy.csv',\
                 index=False)


In [ ]:
# Convert to part of the days:
#Early morning    5 to 8 am
#morning 8 a.m. to 11 a.m
#Late morning     11 am to 12pm
#afternoon 12 pm to 5 pm
#Evening     5 pm to 9 pm
#Night         9 pm to 5 am



